In [1]:
import psycopg2
from psycopg2 import sql
import numpy as np
import pandas as pd
from psycopg2.extensions import adapt, register_adapter, AsIs
from sqlalchemy import create_engine, text
import getpass

In [2]:
# Define the file paths
file_paths = ['2018_Tenure_by_Kitchen.csv', 
              '2019_Tenure_by_Kitchen.csv',
              '2020_Tenure_by_Kitchen.csv', 
              '2021_Tenure_by_Kitchen.csv', 
              '2022_Tenure_by_Kitchen.csv']

# Reading the data files into dataframes
dataframes = [pd.read_csv(file) for file in file_paths]

# Concatenate all the dataframes into one
Tenure_by_Kitchen = pd.concat(dataframes, ignore_index=True)

# Save the merged dataframe to a new CSV file
merged_file_path = 'Tenure_by_Kitchen.csv'
Tenure_by_Kitchen.to_csv(merged_file_path, index=False)

# Provide the path of the merged file
merged_file_path

'Tenure_by_Kitchen.csv'

In [ ]:
database = "f24t03"
user     = input("Type username (pawprint) and hit enter: ")
password = getpass.getpass("Type password and hit enter: ")

connection = psycopg2.connect(database = database,
                              user     = user,
                              host     = 'pgsql',
                              password = password)

Type username (pawprint) and hit enter: jsmm8
Type password and hit enter: ········


In [4]:
with connection, connection.cursor() as cursor:
    cursor.execute("DROP TABLE IF EXISTS tenure_by_kitchen;")
    

    cursor.execute(
        '''
        CREATE TABLE IF NOT EXISTS tenure_by_kitchen (
    geo_id TEXT,
    block_group TEXT,
    census_tract TEXT,
    county TEXT,
    state TEXT,
    est_total NUMERIC,
    moe_total NUMERIC,
    est_own_occ NUMERIC,
    moe_own_occ NUMERIC,
    est_own_occ_kit_complete NUMERIC,
    moe_own_occ_kit_complete NUMERIC,
    est_own_occ_kit_lack NUMERIC,
    moe_own_occ_kit_lack NUMERIC,
    est_rent_occ NUMERIC,
    moe_rent_occ NUMERIC,
    est_rent_occ_kit_complete NUMERIC,
    moe_rent_occ_kit_complete NUMERIC,
    est_rent_occ_kit_lack NUMERIC,
    moe_rent_occ_kit_lack NUMERIC,
    year INT,
    PRIMARY KEY (geo_id, year)
    );
    '''
    )

In [ ]:
# Securely getting the password
mypasswd = getpass.getpass("Enter your database password: ")

# Database connection parameters
username = 'jsmm8'
host = 'pgsql'
database = 'f24t03'

# Constructing the connection string
conn_string = f"postgresql+psycopg2://{username}:{mypasswd}@{host}/{database}"

# Create the SQLAlchemy engine
try:
    engine = create_engine(conn_string)
    print("Database connection established successfully.")
except Exception as conn_err:
    print(f"Database connection error: {conn_err}")

# CSV file to be imported
tenure_by_kitchen = 'Tenure_by_Kitchen.csv'

# Chunk size for batch import
chunk_size = 500

try:
    # Reading and uploading the CSV in chunks
    for chunk in pd.read_csv(tenure_by_kitchen, chunksize=chunk_size):
        # Load the data to PostgreSQL (append data in chunks)
        chunk.to_sql('tenure_by_kitchen', engine, if_exists='append', index=False)
        print(f"Uploaded a chunk of {len(chunk)} records to the database.")
    
    print("CSV data uploaded successfully.")

except pd.errors.EmptyDataError:
    print("The CSV file is empty. Please check the file contents.")
except pd.errors.ParserError as parse_err:
    print(f"Error parsing CSV file: {parse_err}")
except Exception as e:
    print(f"An error occurred: {e}")

Enter your database password: ········
Database connection established successfully.
Uploaded a chunk of 500 records to the database.
Uploaded a chunk of 500 records to the database.
Uploaded a chunk of 500 records to the database.
Uploaded a chunk of 500 records to the database.
Uploaded a chunk of 500 records to the database.
Uploaded a chunk of 500 records to the database.
Uploaded a chunk of 500 records to the database.
Uploaded a chunk of 500 records to the database.
Uploaded a chunk of 500 records to the database.
Uploaded a chunk of 500 records to the database.
Uploaded a chunk of 500 records to the database.
Uploaded a chunk of 500 records to the database.
Uploaded a chunk of 500 records to the database.
Uploaded a chunk of 500 records to the database.
Uploaded a chunk of 500 records to the database.
Uploaded a chunk of 500 records to the database.
Uploaded a chunk of 500 records to the database.
Uploaded a chunk of 500 records to the database.
Uploaded a chunk of 500 records t

In [6]:
# SQL query to grant privileges
grant_privileges_query = "GRANT ALL PRIVILEGES ON TABLE tenure_by_kitchen TO ypd5yb, remcmf, sgdky;"

with engine.connect() as connection:
        connection.execute(text(grant_privileges_query))
        print("Privileges granted successfully.")

Privileges granted successfully.


In [7]:
connection.close()

In [8]:
select_query = "SELECT * FROM tenure_by_kitchen WHERE year = 2022 LIMIT 10;" 

with engine.connect() as connection:
    result = connection.execute(text(select_query))
    df = pd.DataFrame(result.fetchall(), columns=result.keys())
print("Query executed successfully.")
df

Query executed successfully.


,geo_id,block_group,census_tract,county,state,est_total,moe_total,est_own_occ,moe_own_occ,est_own_occ_kit_complete,moe_own_occ_kit_complete,est_own_occ_kit_lack,moe_own_occ_kit_lack,est_rent_occ,moe_rent_occ,est_rent_occ_kit_complete,moe_rent_occ_kit_complete,est_rent_occ_kit_lack,moe_rent_occ_kit_lack,year
0,1500000US290019501001,Block Group 1,Census Tract 9501,Adair County,Missouri,427,74,405,77,405,77,0,12,22,23,22,23,0,12,2022
1,1500000US290019501002,Block Group 2,Census Tract 9501,Adair County,Missouri,612,100,523,89,523,89,0,12,89,52,89,52,0,12,2022
2,1500000US290019501003,Block Group 3,Census Tract 9501,Adair County,Missouri,317,64,275,55,264,54,11,10,42,25,42,25,0,12,2022
3,1500000US290019502001,Block Group 1,Census Tract 9502,Adair County,Missouri,322,70,273,64,273,64,0,12,49,27,49,27,0,12,2022
4,1500000US290019502002,Block Group 2,Census Tract 9502,Adair County,Missouri,378,88,332,79,329,79,3,4,46,36,46,36,0,12,2022
5,1500000US290019502003,Block Group 3,Census Tract 9502,Adair County,Missouri,417,90,345,91,345,91,0,12,72,35,72,35,0,12,2022
6,1500000US290019503001,Block Group 1,Census Tract 9503,Adair County,Missouri,298,76,210,64,210,64,0,12,88,53,88,53,0,12,2022
7,1500000US290019503002,Block Group 2,Census Tract 9503,Adair County,Missouri,272,102,165,76,165,76,0,12,107,76,107,76,0,12,2022
8,1500000US290019503003,Block Group 3,Census Tract 9503,Adair County,Missouri,433,177,60,44,60,44,0,12,373,177,373,177,0,12,2022
9,1500000US290019503004,Block Group 4,Census Tract 9503,Adair County,Missouri,296,108,10,15,10,15,0,12,286,108,286,108,0,12,2022


In [9]:
select_query = "SELECT * FROM tenure_by_kitchen WHERE year = 2018 LIMIT 10;" 

with engine.connect() as connection:
    result = connection.execute(text(select_query))
    df = pd.DataFrame(result.fetchall(), columns=result.keys())
print("Query executed successfully.")
df

Query executed successfully.


,geo_id,block_group,census_tract,county,state,est_total,moe_total,est_own_occ,moe_own_occ,est_own_occ_kit_complete,moe_own_occ_kit_complete,est_own_occ_kit_lack,moe_own_occ_kit_lack,est_rent_occ,moe_rent_occ,est_rent_occ_kit_complete,moe_rent_occ_kit_complete,est_rent_occ_kit_lack,moe_rent_occ_kit_lack,year
0,1500000US290019501001,Block Group 1,Census Tract 9501,Adair County,Missouri,547,71,522,72,498,73,24,24,25,20,25,20,0,11,2018
1,1500000US290019501002,Block Group 2,Census Tract 9501,Adair County,Missouri,466,67,405,61,405,61,0,11,61,41,61,41,0,11,2018
2,1500000US290019501003,Block Group 3,Census Tract 9501,Adair County,Missouri,332,54,293,55,272,54,21,21,39,18,39,18,0,11,2018
3,1500000US290019502001,Block Group 1,Census Tract 9502,Adair County,Missouri,314,50,256,47,245,46,11,15,58,24,53,23,5,8,2018
4,1500000US290019502002,Block Group 2,Census Tract 9502,Adair County,Missouri,331,55,294,52,294,52,0,11,37,20,37,20,0,11,2018
5,1500000US290019502003,Block Group 3,Census Tract 9502,Adair County,Missouri,436,69,347,67,334,64,13,20,89,46,70,51,19,31,2018
6,1500000US290019503001,Block Group 1,Census Tract 9503,Adair County,Missouri,286,70,248,72,236,69,12,18,38,30,38,30,0,11,2018
7,1500000US290019503002,Block Group 2,Census Tract 9503,Adair County,Missouri,272,66,134,46,134,46,0,11,138,66,138,66,0,11,2018
8,1500000US290019503003,Block Group 3,Census Tract 9503,Adair County,Missouri,450,113,113,68,102,68,11,16,337,99,337,99,0,11,2018
9,1500000US290019503004,Block Group 4,Census Tract 9503,Adair County,Missouri,415,94,41,33,41,33,0,11,374,93,374,93,0,11,2018
